# Medical Insurance Cost Prediction - Step 1: Data Loading & Cleaning

This notebook will:
1. Load our insurance data
2. Check for any problems in the data
3. Clean the data for analysis

In [44]:
# Import the tools we need
import pandas as pd  # For working with data tables
import numpy as np   # For math operations
import matplotlib.pyplot as plt  # For basic graphs
import seaborn as sns  # For nicer graphs

# This makes our graphs show up in the notebook
# %matplotlib inline
#type: ignore
%matplotlib inline 

In [45]:
import pandas as pd

# Data Cleaning
df = pd.read_csv('../data/Medical_Insurance_New.csv') # Load CSV File

df = df.iloc[:-2374] # Keep the first 400 rows of data

df = df.drop_duplicates() # Delete duplicates

df = df.dropna() # Delete any rows with empty values

df.to_csv('../data/insurance_cleaned.csv', index=False)


In [46]:
# === DATA QUALITY CHECKS ===
# 1. Check shape (rows, columns)
print("Dataset shape (rows, columns):", insurance_df.shape)

# 2. Check data types
print("\nData types:")
print(insurance_df.dtypes)

# 3. Check missing values
print("\nMissing values per column:")
print(insurance_df.isnull().sum())

# 4. Basic statistics (numeric columns only)
print("\nBasic statistics:")
print(insurance_df.describe())

Dataset shape (rows, columns): (398, 6)

Data types:
age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
dtype: object

Missing values per column:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

Basic statistics:
              age         bmi    children
count  398.000000  398.000000  398.000000
mean    39.525126   30.571897    1.100503
std     14.429124    5.687665    1.194003
min     18.000000   15.960000    0.000000
25%     27.000000   26.605000    0.000000
50%     40.000000   30.350000    1.000000
75%     52.750000   34.430000    2.000000
max     64.000000   49.060000    5.000000
